In [ ]:
from getpass import getpass

password = getpass("Password: ")

Password: ··········


# API


In [ ]:
!pip install requests

In [ ]:
import requests
requests.get("https://10.159.125.33", verify=False, timeout=5)


ConnectTimeout: HTTPSConnectionPool(host='10.159.125.33', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e632b1caa0>, 'Connection to 10.159.125.33 timed out. (connect timeout=5)'))

In [ ]:
import requests
import urllib3

# Deshabilitar advertencias SSL (Colab no tiene tus certificados)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ---------------------------------------------------------
# CONFIGURACIÓN: Cambia estas 2 variables:
# ---------------------------------------------------------

DCE_HOST = "https://10.159.125.33"   # Ejemplo: "https://10.170.203.6"
ACCESS_TOKEN = "3680f766-6976-4051-9fb4-ac57ae433887"      # Pegas tu token generado en la web

# ---------------------------------------------------------
# Hacer PRIMER GET
# ---------------------------------------------------------

url = f"{DCE_HOST}/isxg/v1/devices"

headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

print("🔵 Enviando GET a:", url)

response = requests.get(url, headers=headers, verify=False)

print("Código HTTP:", response.status_code)

if response.status_code == 200:
    print("\n✅ Respuesta OK (primeros 500 caracteres):\n")
    print(response.text[:500])
else:
    print("\n❌ Error:")
    print(response.text)


🔵 Enviando GET a: https://10.159.125.33/isxg/v1/devices


KeyboardInterrupt: 

In [ ]:
import requests
import json
from datetime import datetime, timedelta

# ---------------------------------------------------------
# CONFIGURACIÓN
# ---------------------------------------------------------
DCE_HOST = "https://TU_IP_O_HOST_DCE"   # Ej: https://10.10.10.5
USERNAME = "TU_USUARIO"
PASSWORD = "TU_PASSWORD"

TARGET_GROUP_NAME = "IDEO"       # Grupo final que quieres leer
TARGET_LOCATION = "CALi"         # Esto ayuda a filtrar si hay varios IDEO
# ---------------------------------------------------------

access_token = None
token_expiration = None

# ---------------------------------------------------------
# 1. Obtener token
# ---------------------------------------------------------
def get_token():
    global access_token, token_expiration

    url = f"{DCE_HOST}/oauth/token"
    data = {
        "username": USERNAME,
        "password": PASSWORD,
        "grant_type": "password"
    }

    print("🟦 Solicitando token...")
    response = requests.post(url, data=data, verify=False)

    if response.status_code != 200:
        print("❌ ERROR token:", response.text)
        exit()

    token_data = response.json()
    access_token = token_data["access_token"]
    token_expiration = datetime.now() + timedelta(minutes=4)

    print("🔑 Token obtenido exitosamente\n")

# ---------------------------------------------------------
# 2. Llamada autenticada
# ---------------------------------------------------------
def api_get(endpoint):
    global access_token

    # Renueva token si está expirado
    if access_token is None or datetime.now() > token_expiration:
        get_token()

    headers = {"Authorization": f"Bearer {access_token}"}

    url = f"{DCE_HOST}{endpoint}"

    response = requests.get(url, headers=headers, verify=False)

    if response.status_code != 200:
        print("❌ ERROR llamada:", response.text)
        exit()

    return response.json()

# ---------------------------------------------------------
# 3. Buscar un grupo por nombre
# ---------------------------------------------------------
def find_group_by_name(name):
    groups = api_get("/isxg/v1/deviceGroups")

    for g in groups:
        if g["name"].lower() == name.lower():
            return g

    return None

# ---------------------------------------------------------
# 4. Obtener dispositivos de un grupo
# ---------------------------------------------------------
def get_devices_of_group(group_id):
    return api_get(f"/isxg/v1/devices?groupId={group_id}")

# ---------------------------------------------------------
# 5. Obtener sensores de un dispositivo
# ---------------------------------------------------------
def get_sensors(device_id):
    return api_get(f"/isxg/v1/sensors?deviceId={device_id}")

# ---------------------------------------------------------
# PROGRAMA PRINCIPAL
# ---------------------------------------------------------
if __name__ == "__main__":
    print("\n=========== CONSULTA DCE ===========")

    # 1. Obtener token inicial
    get_token()

    # 2. Buscar el grupo IDEO
    print("🔍 Buscando grupo:", TARGET_GROUP_NAME)
    group = find_group_by_name(TARGET_GROUP_NAME)

    if group is None:
        print("❌ No se encontró el grupo IDEO")
        exit()

    print("📂 Grupo encontrado → ID:", group["id"], "\n")

    # 3. Obtener dispositivos del grupo IDEO
    devices = get_devices_of_group(group["id"])

    if len(devices) == 0:
        print("❌ No hay dispositivos en este grupo")
        exit()

    print(f"🖥️ Dispositivos encontrados: {len(devices)}")

    # Tomo el primero (o el único)
    device = devices[0]
    device_id = device["id"]

    print("➡️  Usando dispositivo:")
    print("   Hostname:", device["hostname"])
    print("   IP:", device["ipAddress"])
    print("   ID:", device_id, "\n")

    # 4. Obtener sensores
    print("📡 Descargando sensores...")
    sensors = get_sensors(device_id)

    print(f"📊 Sensores recibidos: {len(sensors)}\n")

    for s in sensors:
        print(f"- {s['name']}: {s.get('value', 'N/A')} (Estado: {s.get('status', 'N/A')})")


# Lectura de datos

In [1]:
import pandas as pd

Excel = "datos_DCE.xlsx"

ml = pd.read_excel(Excel, sheet_name="ML")
r1 = pd.read_excel(Excel, sheet_name="RECT1")
r2 = pd.read_excel(Excel, sheet_name="RECT2")
tr = pd.read_excel(Excel, sheet_name="TR")


In [2]:
print(ml.head())

                Fecha  ANALOG INPUT - ML CURRENT AC S  \
0 2025-11-24 10:00:00                           59.46   

   ANALOG INPUT - ML CURRENT AC T  ANALOG INPUT - °C SALA S01  \
0                           66.25                        23.0   

   ANALOG INPUT - ML VOLTAGE AC R-S  ANALOG INPUT - ML VOLTAGE AC S-T  \
0                            214.93                            214.35   

   08 - CORRIENTE DC DE LAS BATERIAS Link Status  \
0                                0.0      Online   

   ANALOG INPUT - ML VOLTAGE AC T-R  ANALOG INPUT - ML CURRENT AC R  \
0                            219.85                           58.46   

  DIGITAL OUTPUT - ML AC FAIL  ANALOG INPUT - °C SALA S02  \
0                    INACTIVO                        21.0   

  13 - TEST DE BATERIAS  
0         DESHABILITADO  


In [ ]:
ml.columns = (
    ml.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "")
    .str.replace("__", "_")
)

print(ml.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   fecha                            1 non-null      datetime64[ns]
 1   analog_input_ml_current_ac_s     1 non-null      float64       
 2   analog_input_ml_current_ac_t     1 non-null      float64       
 3   analog_input_°c_sala_s01         1 non-null      float64       
 4   analog_input_ml_voltage_ac_rs    1 non-null      float64       
 5   analog_input_ml_voltage_ac_st    1 non-null      float64       
 6   08_corriente_dc_de_las_baterias  1 non-null      float64       
 7   link_status                      1 non-null      object        
 8   analog_input_ml_voltage_ac_tr    1 non-null      float64       
 9   analog_input_ml_current_ac_r     1 non-null      float64       
 10  digital_output_ml_ac_fail        1 non-null      object        
 1

In [ ]:


r1.columns = (
    r1.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "")
    .str.replace("__", "_")
)

print("RECTIFICADOR 1:")
#print(r1.head())
print(r1.info())

RECTIFICADOR 1:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   fecha                               1 non-null      datetime64[ns]
 1   07_rectificadores_instalados        1 non-null      float64       
 2   05_modo_del_sistema                 1 non-null      object        
 3   09_corriente_dc_de_las_baterias     1 non-null      float64       
 4   14_estado_de_la_ac                  1 non-null      object        
 5   19_estado_fusibles_de_las_baterias  1 non-null      object        
 6   01_voltaje_ac_del_sistema           1 non-null      float64       
 7   10_temperatura_de_las_baterias      1 non-null      float64       
 8   15_estado_de_los_rectificadores     1 non-null      object        
 9   link_status                         1 non-null      object        
 10  17_estado_fusi

In [ ]:
r2.columns = (
    r2.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "")
    .str.replace("__", "_")
)

print("RECTIFICADOR 2:")
#print(r2.head())
print(r2.info())

RECTIFICADOR 2:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   fecha                               1 non-null      datetime64[ns]
 1   04_porcentaje_de_carga_del_sistema  1 non-null      float64       
 2   16_estado_de_la_carga               1 non-null      object        
 3   15_estado_de_los_rectificadores     1 non-null      object        
 4   18_estado_de_las_baterias           1 non-null      object        
 5   20_estado_lvd_de_las_baterias       1 non-null      object        
 6   08_rectificadores_fallados          1 non-null      float64       
 7   09_corriente_dc_de_las_baterias     1 non-null      float64       
 8   05_modo_del_sistema                 1 non-null      object        
 9   link_status                         1 non-null      object        
 10  06_numero_de_f

In [ ]:
# Convertir datos numéricos
'''
def clean_numeric(col):
    return (
        col.astype(str)
        .str.replace(",", ".")
        .str.extract(r"([-+]?\d*\.?\d+)")[0]
        .astype(float)
    )

for col in ml.select_dtypes(include="object").columns:
    try:
        ml[col] = clean_numeric(ml[col])
    except:
        pass

  '''

# Mapeo

In [3]:
import pandas as pd

In [4]:
class Tablero_principal:
    def __init__(self, df):
        # Guardamos copia del DataFrame
        self.df = df.copy()

        # Normalizamos nombres de columnas
        self.df.columns = (
            self.df.columns
            .str.lower()
            .str.replace(" ", "_")
            .str.replace("-", "")
            .str.replace("__", "_")
        )

        # Convertimos fecha si existe
        if "fecha" in self.df.columns:
            self.df["fecha"] = pd.to_datetime(self.df["fecha"], errors="coerce")

    # -------------------------------------------------------------------
    def ver(self, n=5):
        """Ver primeras filas del ML."""
        return self.df.head(n)

    # -------------------------------------------------------------------
    def columnas(self):
        """Listar columnas disponibles."""
        return list(self.df.columns)

    # -------------------------------------------------------------------
    def ultimo_estado(self):
        """Obtener última fila (por fecha si existe)."""
        if "fecha" in self.df.columns:
            return self.df.sort_values("fecha").iloc[-1]
        return self.df.iloc[-1]

    # -------------------------------------------------------------------
    def filtrar_fecha(self, inicio, fin):
        """Filtrar datos por rango de fecha."""
        if "fecha" not in self.df.columns:
            raise ValueError("No existe columna 'fecha'.")
        return self.df[(self.df["fecha"] >= inicio) & (self.df["fecha"] <= fin)]

    # -------------------------------------------------------------------
    def to_object(self, row=None):
        """
        Convierte una fila del DataFrame en un objeto MLPanel (tu clase original).
        Si no se pasa row, usa la última fila.
        """
        if row is None:
            row = self.ultimo_estado()
        return Tablero_principal(row)


In [5]:
class Rectificador:
    def __init__(self, df):
        # Guardar dataframe original
        self.df = df.copy()

        # Normalizar nombres de columnas
        self.df.columns = (
            self.df.columns
            .str.lower()
            .str.replace(" ", "_")
            .str.replace("-", "")
            .str.replace("__", "_")
        )

        # Convertir fecha si existe
        if "fecha" in self.df.columns:
            self.df["fecha"] = pd.to_datetime(self.df["fecha"], errors="coerce")

    # Mostrar primeras filas
    def ver(self, n=5):
        return self.df.head(n)

    # Listar columnas disponibles
    def columnas(self):
        return list(self.df.columns)

    # Obtener una columna específica
    def get(self, nombre):
        nombre = nombre.lower()
        return self.df[nombre] if nombre in self.df.columns else None

    # Filtrar por rangos de fecha
    def filtrar_fecha(self, inicio, fin):
        if "fecha" not in self.df.columns:
            raise ValueError("La hoja no tiene columna 'fecha'.")
        return self.df[(self.df["fecha"] >= inicio) & (self.df["fecha"] <= fin)]

    # Obtener fila más reciente
    def ultimo_estado(self):
        if "fecha" in self.df.columns:
            return self.df.sort_values("fecha").iloc[-1]
        return self.df.iloc[-1]


In [6]:
class transferencia:
    def __init__(self, df):
        # Guardar dataframe original
        self.df = df.copy()

        # Normalizar nombres de columnas
        self.df.columns = (
            self.df.columns
            .str.lower()
            .str.replace(" ", "_")
            .str.replace("-", "")
            .str.replace("__", "_")
        )

        # Convertir fecha si existe
        if "fecha" in self.df.columns:
            self.df["fecha"] = pd.to_datetime(self.df["fecha"], errors="coerce")

    # Mostrar primeras filas
    def ver(self, n=5):
        return self.df.head(n)

    # Listar columnas disponibles
    def columnas(self):
        return list(self.df.columns)

    # Obtener una columna específica
    def get(self, nombre):
        nombre = nombre.lower()
        return self.df[nombre] if nombre in self.df.columns else None

    # Filtrar por rangos de fecha
    def filtrar_fecha(self, inicio, fin):
        if "fecha" not in self.df.columns:
            raise ValueError("La hoja no tiene columna 'fecha'.")
        return self.df[(self.df["fecha"] >= inicio) & (self.df["fecha"] <= fin)]

    # Obtener fila más reciente
    def ultimo_estado(self):
        if "fecha" in self.df.columns:
            return self.df.sort_values("fecha").iloc[-1]
        return self.df.iloc[-1]

In [7]:
r1_nodo = Rectificador(r1)
r2_nodo = Rectificador(r2)
ml_nodo = Tablero_principal(ml)
tr_nodo = Tablero_principal(tr)

In [8]:
print(ml_nodo.ultimo_estado())

fecha                              2025-11-24 10:00:00
analog_input_ml_current_ac_s                     59.46
analog_input_ml_current_ac_t                     66.25
analog_input_°c_sala_s01                          23.0
analog_input_ml_voltage_ac_rs                   214.93
analog_input_ml_voltage_ac_st                   214.35
08_corriente_dc_de_las_baterias                    0.0
link_status                                     Online
analog_input_ml_voltage_ac_tr                   219.85
analog_input_ml_current_ac_r                     58.46
digital_output_ml_ac_fail                     INACTIVO
analog_input_°c_sala_s02                          21.0
13_test_de_baterias                      DESHABILITADO
Name: 0, dtype: object


In [9]:
#print(r1_nodo.ver())
#print(r1_nodo.columnas())

# Obtener voltaje DC del sistema
#print(r1_nodo.get("01_voltaje_ac_del_sistema"))

# Último registro
print(r1_nodo.ultimo_estado())

fecha                                 2025-11-24 10:00:00
07_rectificadores_instalados                         21.0
05_modo_del_sistema                                 Float
09_corriente_dc_de_las_baterias                       0.0
14_estado_de_la_ac                                 Normal
19_estado_fusibles_de_las_baterias                 Normal
01_voltaje_ac_del_sistema                           213.0
10_temperatura_de_las_baterias                       26.0
15_estado_de_los_rectificadores                    Normal
link_status                                        Online
17_estado_fusibles_de_la_carga                     Normal
06_numero_de_fases                                    3.0
03_corriente_dc_del_sistema                         119.0
18_estado_de_las_baterias                          Normal
08_rectificadores_fallados                            0.0
12_estado_del_sistema                              Normal
02_voltaje_dc_del_sistema                           54.47
13_estado_cont

In [10]:
print(r2_nodo.ultimo_estado())

fecha                                 2025-11-24 10:00:00
04_porcentaje_de_carga_del_sistema                   14.0
16_estado_de_la_carga                              Normal
15_estado_de_los_rectificadores                    Normal
18_estado_de_las_baterias                          Normal
20_estado_lvd_de_las_baterias                      Normal
08_rectificadores_fallados                            0.0
09_corriente_dc_de_las_baterias                       0.0
05_modo_del_sistema                                 Float
link_status                                        Online
06_numero_de_fases                                    3.0
19_estado_fusibles_de_las_baterias                 Normal
07_rectificadores_instalados                         19.0
01_voltaje_ac_del_sistema                           211.0
11_corriente_dc_de_la_carga                         157.0
13_estado_control_del_sistema                      Normal
14_estado_de_la_ac                                 Normal
02_voltaje_dc_

In [12]:
print(tr_nodo.ultimo_estado())

fecha                                2025-12-02 15:30:00
08_voltaje_ac_del_generador_l3l1                     0.0
11_corriente_ac_de_la_carga_l1                      58.0
19_ac_fail                                      INACTIVO
18_modo_del_control                           AUTOMATICO
05_voltaje_de_la_bateria                            27.5
15_potencia_reactiva_de_la_carga                     -40
09_frencuencia_del_generador                         0.0
22_contactor_del_generador                      INACTIVO
20_contactor_de_red                               ACTIVO
21_generador_encendido                          INACTIVO
03_voltaje_ac_del_sistema_l3l1                     215.0
07_voltaje_ac_del_generador_l2l3                     0.0
14_potencia_activa_de_la_carga                      21.0
17_factor_de_potencia_de_la_carga                   0.98
13_corriente_ac_de_la_carga_l3                      59.0
23_alarma                                       INACTIVO
16_potencia_aparente_de_la_carg

In [11]:
import datetime

def to_unix_ms(year, month, day):
    dt = datetime.datetime(year, month, day, 0, 0, 0)
    return int(dt.timestamp() * 1000)

start = to_unix_ms(2025, 11, 21)
end = to_unix_ms(2025, 11, 22)

print("startTime:", start)
print("endTime:", end)


startTime: 1763683200000
endTime: 1763769600000
